## Nepali Character Recognition
***<div style="text-align: right">[TOTAL SCORE: 20]</div>***

For this project, we are going to implement Nepali Character Recognition. The dataset is provided in the path: ./consonants. The final model needs to recognize the 36 consonants of the nepali alphabets (क,ख,ग....ज्ञ). We have taken the dataset from [here](https://www.kaggle.com/ashokpant/devanagari-character-dataset). You can explore more with the data, however, for this assignment you need to classify only the 36 characters.

Let us first import the packages.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
# import cv2
%matplotlib inline

### Import any other libraries as required
# YOUR CODE HERE
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping
# raise NotImplementedError()

Using TensorFlow backend.


We need to classify 36 characters.

In [2]:
CATEGORIES = ["क","ख","ग","घ","ङ","च","छ","ज","झ","ञ","ट","ठ","ड","ढ","ण","त","थ","द","ध","न","प","फ","ब","भ","म","य","र","ल","व","श","ष","स","ह","क्ष","त्र","ज्ञ"]

### Exercise 1:  Loading, Resizing and Viewing Images
***<div style="text-align: right">[SCORE: 1]</div>***

1. Read the dataset.pickle to data
2. Explore the dataset

The data consist of 30 X 30 Resized images

In [3]:
filepath = 'dataset.pickle'
data = None
# YOUR CODE HERE
data = pd.read_pickle(filepath)
data = pd.DataFrame(data)
data.columns = ["image", "label"]
# print(data["label"].unique())
data.head()
# raise NotImplementedError()

,image,label
0,"[[255, 248, 253, 255, 251, 253, 254, 236, 220,...",क
1,"[[253, 253, 254, 240, 249, 250, 246, 250, 255,...",क
2,"[[212, 202, 157, 120, 104, 53, 21, 91, 183, 20...",क
3,"[[255, 251, 247, 246, 249, 253, 255, 255, 255,...",क
4,"[[247, 254, 243, 243, 241, 232, 213, 216, 235,...",क


In [4]:
#### INTENTIONALLY LEFT BLANK####

### Exercise 2: Normalise the Data
***<div style="text-align: right">[SCORE: 2]</div>***

- Normalise the Image Pixels
- Load all the features(30X30 images) to X and all the targets to y

In [5]:
data["image"] = data["image"].apply(lambda X:X.reshape(900))
X = data["image"].values
empty_x = np.empty(shape = (X.shape[0],30,30))
for i in range(0,X.shape[0]):
    empty_x[i,:,:] = X[i].reshape((30,30))

In [6]:
X = None
y = None

y = data["label"]
X = empty_x/255.0;
X = X.reshape(*X.shape, 1)
# YOUR CODE HERE
# raise NotImplementedError()

In [7]:
#### INTENTIONALLY LEFT BLANK####

### Exercise 3: Convert target to one hot encoded variable
***<div style="text-align: right">[SCORE: 1]</div>***
Convert target to one hot encoded variable

In [8]:
Y = None  # encoded Variable
# YOUR CODE HERE
label_y = LabelEncoder().fit_transform(y)
Y = OneHotEncoder(sparse=False).fit_transform(label_y.reshape(-1,1))
# raise NotImplementedError()

In [9]:
#### INTENTIONALLY LEFT BLANK####

### Exercise 4: Train-Test Split
***<div style="text-align: right">[SCORE: 2]</div>***
Split the Data into Train and Test, don't forget to shuffle the data as well as stratify while splitting.
<br>Use the given random_state and test_size

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = None, None,None,None
random_state = 2019
test_size = 0.33
# YOUR CODE HERE
IMG_ROWS = IMG_COLS = 30
NUM_CLASSES = 36
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=random_state, stratify=y)
# raise NotImplementedError()

In [11]:
#### INTENTIONALLY LEFT BLANK####

### Exercise 5: Create and Compile your Model
***<div style="text-align: right">[SCORE: 4]</div>***
Create your network architecture using Keras below and compile it.
<br>Don't forget to include Conv2D and MaxPooling
<br>Use categorical crossentropy


In [12]:
model = None

# YOUR CODE HERE
model = Sequential()
model.add(Conv2D(32, (3,3), padding="same", activation="relu", input_shape=(IMG_ROWS, IMG_COLS, 1)))
model.add(BatchNormalization()) 
model.add(MaxPooling2D())
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(NUM_CLASSES, activation="softmax"))
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 15, 64)        256       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        36928     
__________

In [13]:
#### INTENTIONALLY LEFT BLANK####

### Exercise 5: Setup Callback Functions
***<div style="text-align: right">[SCORE: 1]</div>***
Save your best model Checkpoint into checkpoint_path
<br> You can use other callback functions for early stopping

In [14]:
checkpoint_path = 'best_model.h5'
# YOUR CODE HERE
callbacks = [EarlyStopping(monitor = 'val_loss', patience = 3),
            ModelCheckpoint(checkpoint_path, save_best_only = True, verbose = 1, monitor="val_acc")]

In [15]:
#### INTENTIONALLY LEFT BLANK####

### Exercise 6: Train your model
***<div style="text-align: right">[SCORE: 9]</div>***
- Fit your Model saving the best model
- Comment out your training section and load your model
- Plot the History
- Evaluate with the test set created previously
<br> Your SCORE depends on your model accuracy

In [16]:
# Train your model

# YOUR CODE HERE
NO_EPOCHS = 25
BATCH_SIZE = 32
# History = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=BATCH_SIZE, epochs=NO_EPOCHS, callbacks=callbacks)

Train on 4944 samples, validate on 2436 samples
Epoch 1/25
4944/4944 [==============================] - 71s 14ms/step - loss: 2.1551 - acc: 0.4221 - val_loss: 0.8879 - val_acc: 0.7410

Epoch 00001: val_acc improved from -inf to 0.74097, saving model to best_model.h5
Epoch 2/25
4944/4944 [==============================] - 70s 14ms/step - loss: 0.9506 - acc: 0.7118 - val_loss: 0.6165 - val_acc: 0.8144

Epoch 00002: val_acc improved from 0.74097 to 0.81445, saving model to best_model.h5
Epoch 3/25
4944/4944 [==============================] - 69s 14ms/step - loss: 0.6615 - acc: 0.7959 - val_loss: 0.6468 - val_acc: 0.8017

Epoch 00003: val_acc did not improve from 0.81445
Epoch 4/25
4944/4944 [==============================] - 68s 14ms/step - loss: 0.5067 - acc: 0.8434 - val_loss: 0.4726 - val_acc: 0.8518

Epoch 00004: val_acc improved from 0.81445 to 0.85181, saving model to best_model.h5
Epoch 5/25
4944/4944 [==============================] - 67s 14ms/step - loss: 0.4109 - acc: 0.8693 - v

In [19]:
# Load your model and evaluate in your test set
score = None
# YOUR CODE HERE
train_model = model.load_weights(checkpoint_path)
score = model.evaluate(X_test,y_test)
print('Test score:', score[0])
print('Test accuracy:', score[1])

2436/2436 [==============================] - 9s 4ms/step
Test score: 0.3543261014741703
Test accuracy: 0.9014778325123153


In [ ]:
#### INTENTIONALLY LEFT BLANK####

In [ ]:
#### INTENTIONALLY LEFT BLANK####

In [ ]:
#### INTENTIONALLY LEFT BLANK####

In [ ]:
#### INTENTIONALLY LEFT BLANK####

In [ ]:
#### INTENTIONALLY LEFT BLANK####
### Congratulations ! You have Completed the Project 